# Bumble Recommender Review

In [1]:
!ls

07. Control Flow.ipynb      Solutions.ipynb
08. Functions.ipynb         aboutyou_qs.ipynb
Conversational.ipynb        bumble_passionproject.ipynb
Interview_cake_python.ipynb data
README.md                   jm_review.ipynb
Reviews.csv


In [2]:
import pandas as pd
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns
import time
%matplotlib inline

In [3]:
#Import surprise libraries
from surprise import Reader, Dataset
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import SVD, baseline_only
from surprise.prediction_algorithms import KNNWithMeans, KNNBasic, KNNBaseline
from surprise.model_selection import GridSearchCV
from surprise import accuracy, dataset
from surprise.dataset import DatasetAutoFolds

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
pd.set_option('display.max_columns', None)

In [7]:
df = pd.read_csv("Reviews.csv")

In [9]:

df.head(3)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...


In [10]:
df.dropna(inplace=True)

In [22]:
# Take a subset of the data for baseline to speed computation
sample = df.sample(n=5000)
sample.shape

(5000, 10)

In [55]:
# Filter the dataset for the columns that can be used for pairwise comparisons
ratings = sample[["UserId", "ProductId", "Score"]]
ratings.head()

,UserId,ProductId,Score
236730,A3NFWMZ650JPBL,B003IN848Q,5
426135,AZZP14UZ813US,B003SE8QYC,5
258917,A3TWUOM0X7V72V,B004S719JM,5
153648,A2PUDQ50F2T1GB,B001E5E10A,5
157292,AC57ZRGZMHNR8,B0019QT66I,2


In [56]:
ratings.shape

(5000, 3)

In [57]:
ratings.Score.value_counts()

5    3207
4     676
1     500
3     360
2     257
Name: Score, dtype: int64

In [58]:
# class DataSet(dataset.DatasetAutoFolds):
#     #Creates data set that can be accessed by Surprise including folds for cross validation
    
#     def __init__(self, df, reader):
        
        
#         self.raw_ratings = [(uid, iid, r, None) for (uid, iid, r) in zip(df['UserId'], 
#                                                                          df['ProductId'], df['Score'])]
#         self.reader=reader

In [59]:
# Instantiate an instance of surprise Reader to enable surprise to read the ratings
reader = Reader(rating_scale=(0,5))

In [60]:
# Instantiate and instance of surprise Dataset with our ratings data and the above reader

#### NOTE have now removed the Dataset class so this is calling the surprise Dataset !
data = DataSet(ratings, reader)

In [61]:
# Extract raw ratings from the dataset
raw_ratings = data.raw_ratings

In [62]:
# take a look at the raw ratings
raw_ratings

[('A3NFWMZ650JPBL', 'B003IN848Q', 5, None),
 ('AZZP14UZ813US', 'B003SE8QYC', 5, None),
 ('A3TWUOM0X7V72V', 'B004S719JM', 5, None),
 ('A2PUDQ50F2T1GB', 'B001E5E10A', 5, None),
 ('AC57ZRGZMHNR8', 'B0019QT66I', 2, None),
 ('AQGW2YBHZH86G', 'B000EDK6FM', 5, None),
 ('A1SGS3TIPTMHWI', 'B000V6FU0I', 3, None),
 ('A1UASXJKH475MV', 'B007M832YY', 5, None),
 ('A393S80SDOORH6', 'B0006VB3S6', 4, None),
 ('A3Q9HY0LU7MN6E', 'B004YV80O4', 4, None),
 ('A1JFN3M58B8A0Y', 'B000NIFO96', 4, None),
 ('A2DC7N8B6J36X4', 'B000ICPUYQ', 1, None),
 ('A13MB2B1MS0X94', 'B0006Z7NOK', 5, None),
 ('A1TC7NGPD6ERGK', 'B0017OGFU2', 5, None),
 ('A2TRB93M121ZJX', 'B001P3NU4O', 2, None),
 ('A1XQ3B8VQQV3UO', 'B0013T71K4', 5, None),
 ('A1NCM5LAXZ04W2', 'B002AQYN3W', 4, None),
 ('A2WP4176XSV5C2', 'B0000CFH6P', 4, None),
 ('A1HFHEYXF3ETFL', 'B000E46LXM', 5, None),
 ('AIKP8E0QTP0K9', 'B000TIXX4I', 5, None),
 ('AZYMD9P9F9UZ6', 'B000PMC7GI', 4, None),
 ('A1Z4WPIT9QZ9Z9', 'B002YRBATC', 1, None),
 ('A62KCY1O252N1', 'B000FYXBE8', 4, N

### Split into training and test sets

In [63]:
# Our training test split will be 80/20, first we define this threshold
test_threshold = int(.8 * len(raw_ratings))

# Create the train set
train_raw_ratings = raw_ratings[:test_threshold] 

# Create the test set
test_raw_ratings = raw_ratings[test_threshold:] 

In [64]:
# We'll also create a validation set at 20% of the initial training set
val_threshold = int(.8 * len(train_raw_ratings))
val_raw_ratings = train_raw_ratings[val_threshold:] #create the validation set
train_raw_ratings = train_raw_ratings[:val_threshold] #re_assign the training set

In [65]:
# Make the training set the data
data.raw_ratings = train_raw_ratings

### Collaborative Filtering

In [66]:
def Knn_Basic(data, user, item):
    '''
    Function to run different similarity metrics across KNNBasic method. 
    
    '''
    frame = []
    similarity_met = ['cosine', 'msd', 'pearson']
    user_item= [True, False]
    for i in similarity_met:
            #user-user similarities
            print("Evaluation of {} similarity for KNNBasic {} comparison: ". format(i, user))
            results = cross_validate(KNNBasic(sim_options={'name': i, 'user_based': True}), 
                           data=data, cv=5, return_train_measures=True, n_jobs=-1, verbose = True)
            print('\n\n')
            
            
            
            #item-item similarities
            print("Evaluation of {} similarity for KNNBasic {} comparison: ". format(i, item))
            results = cross_validate(KNNBasic(sim_options={'name': i, 'user_based': False}), 
                           data=data, cv=5, return_train_measures=True, n_jobs=-1, verbose = True)
            print('\n\n')
            
            
    return None

In [67]:
Knn_Basic(data, 'user-user', 'item-item')

Evaluation of cosine similarity for KNNBasic user-user comparison: 
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.3762  1.3658  1.2992  1.3642  1.3136  1.3438  0.0311  
MAE (testset)     1.0811  1.1092  1.0538  1.0785  1.0562  1.0758  0.0201  
RMSE (trainset)   0.4671  0.4528  0.4656  0.4433  0.4601  0.4578  0.0088  
MAE (trainset)    0.1532  0.1442  0.1520  0.1383  0.1472  0.1470  0.0054  
Fit time          0.22    0.18    0.14    0.13    0.14    0.16    0.03    
Test time         0.00    0.01    0.00    0.00    0.00    0.00    0.00    



Evaluation of cosine similarity for KNNBasic item-item comparison: 
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.3018  1.4164  1.4023  1.3691  1.2246  1.3428  0.0712  
MAE (testset)     1.0511  1.1299  1.1216  1.0799  0.9964  1.0758  0.

In [68]:
def Knn_Baseline(data, user, item):
    '''
    Function to run different similarity metrics across KNNBaseline method. 
    
    '''
    frame = []
    similarity_met = ['cosine', 'msd', 'pearson']
    user_item= [True, False]
    for i in similarity_met:
            #user-user similarities
            print("Evaluation of {} similarity for KNNBaseline {} comparison: ". format(i, user))
            results = cross_validate(KNNBaseline(sim_options={'name': i, 'user_based': True}), 
                           data=data, cv=5, return_train_measures=True, n_jobs=-1, verbose = True)
            print('\n\n')
            
            
            
            #item-item similarities
            print("Evaluation of {} similarity for KNNBaseline {} comparison: ". format(i, item))
            results = cross_validate(KNNBaseline(sim_options={'name': i, 'user_based': False}), 
                           data=data, cv=5, return_train_measures=True, n_jobs=-1, verbose = True)
            print('\n\n')
            
            
    return None

In [69]:
Knn_Baseline(data, 'user-user', 'item-item')

Evaluation of cosine similarity for KNNBaseline user-user comparison: 
Evaluating RMSE, MAE of algorithm KNNBaseline on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.2925  1.3960  1.3086  1.3726  1.3307  1.3401  0.0388  
MAE (testset)     1.0436  1.1077  1.0541  1.1030  1.0482  1.0713  0.0280  
RMSE (trainset)   0.4457  0.4441  0.4206  0.3943  0.4307  0.4271  0.0188  
MAE (trainset)    0.1465  0.1512  0.1349  0.1220  0.1369  0.1383  0.0101  
Fit time          0.15    0.15    0.15    0.15    0.15    0.15    0.00    
Test time         0.00    0.00    0.00    0.00    0.00    0.00    0.00    



Evaluation of cosine similarity for KNNBaseline item-item comparison: 
Evaluating RMSE, MAE of algorithm KNNBaseline on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.2877  1.3406  1.3704  1.3789  1.3287  1.3413  0.0325  
MAE (testset)     1.0434  1.0655  1.0899  1.1027  1.0634